# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [20]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,polyarnyy,69.1989,33.4478,2.16,99,100,9.23,RU,1709763764
1,1,scottburgh,-30.2867,30.7532,21.69,73,100,1.55,ZA,1709763764
2,2,grytviken,-54.2811,-36.5092,6.56,60,100,0.49,GS,1709763764
3,3,key west,24.5557,-81.7826,27.00,81,0,3.09,US,1709763765
4,4,albany,42.6001,-73.9662,10.77,94,100,0.69,US,1709763765


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [21]:
%%capture --no-display

# Configure the map plot
humid_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", alpha=0.6,
    hover_cols = ["City", "Humidity"]
)

# Display the map plot
humid_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values

# Set criteria
vacay_max_humid = city_data_df['Humidity']<70
vacay_clouds = city_data_df['Cloudiness']<99
vacay_temp_min = city_data_df['Max Temp']>10
vacay_temp_max = city_data_df['Max Temp']<27
vacay_wind_max = city_data_df['Wind Speed']<20

# Narrow down cities that fit criteria
vacay_cities_df = city_data_df[vacay_clouds & vacay_temp_max & vacay_temp_min & vacay_max_humid & vacay_wind_max]

# Drop any rows with null values
vacay_cities_df = vacay_cities_df.dropna()

# Display sample data
vacay_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,margaret river,-33.9500,115.0667,18.38,68,2,4.63,AU,1709763772
29,29,cidade velha,14.9167,-23.6167,22.26,68,20,8.75,CV,1709763774
36,36,leteri,24.0500,77.4000,17.18,30,3,3.03,IN,1709763776
39,39,calvinia,-31.4707,19.7760,18.15,35,16,1.94,ZA,1709763777
40,40,puerto natales,-51.7236,-72.4875,10.15,67,94,1.63,CL,1709763778
...,...,...,...,...,...,...,...,...,...,...
543,543,concepcion,-36.8270,-73.0498,18.89,52,0,7.20,CL,1709764035
544,544,hassi messaoud,31.6804,6.0729,13.92,38,0,0.51,DZ,1709764035
546,546,prairie du chien,43.0517,-91.1412,13.05,19,0,5.14,US,1709764036
557,557,allendale,42.9723,-85.9537,12.26,39,4,1.34,US,1709763828


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame({
    "City" : vacay_cities_df["City"], 
    'Country' : vacay_cities_df['Country'], 
    "Lat" : vacay_cities_df['Lat'], 
    "Lng" : vacay_cities_df["Lng"], 
    "Humidity" : vacay_cities_df["Humidity"]
                })

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,margaret river,AU,-33.9500,115.0667,68,
29,cidade velha,CV,14.9167,-23.6167,68,
36,leteri,IN,24.0500,77.4000,30,
39,calvinia,ZA,-31.4707,19.7760,35,
40,puerto natales,CL,-51.7236,-72.4875,67,
...,...,...,...,...,...,...
543,concepcion,CL,-36.8270,-73.0498,52,
544,hassi messaoud,DZ,31.6804,6.0729,38,
546,prairie du chien,US,43.0517,-91.1412,19,
557,allendale,US,42.9723,-85.9537,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
#SOPHIE SOMETHING IS WRONG HERE

# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
    "filter": '',
    "bias": ''}

# Print a message to follow up the hotel search
print("Your hotel search has begun. . .")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    url = "https://api.geoapify.com/v2/places&APPID=" + geoapify_key


    # Make and API request using the params dictionaty
    name_address = requests.get(url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Your hotel search has begun. . .
margaret river - nearest hotel: No hotel found
cidade velha - nearest hotel: No hotel found
leteri - nearest hotel: No hotel found
calvinia - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
crossett - nearest hotel: No hotel found
north bend - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
bamora - nearest hotel: No hotel found
heber springs - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
najran - nearest hotel: No hotel found
stephenville - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
ponta delgada - nearest hotel: No hotel found
kerikeri - nearest hotel: No hotel found
blenheim - nearest hotel: No hotel found
idfu - nearest hotel: No hotel found
sao borja - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
25,margaret river,AU,-33.9500,115.0667,68,No hotel found
29,cidade velha,CV,14.9167,-23.6167,68,No hotel found
36,leteri,IN,24.0500,77.4000,30,No hotel found
39,calvinia,ZA,-31.4707,19.7760,35,No hotel found
40,puerto natales,CL,-51.7236,-72.4875,67,No hotel found
...,...,...,...,...,...,...
543,concepcion,CL,-36.8270,-73.0498,52,No hotel found
544,hassi messaoud,DZ,31.6804,6.0729,38,No hotel found
546,prairie du chien,US,43.0517,-91.1412,19,No hotel found
557,allendale,US,42.9723,-85.9537,39,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", alpha=0.6,
    hover_cols = ["City", "Humidity", "Country", "Hotel Name"]
)
# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)